<a href="https://colab.research.google.com/github/AnjaliAleti/Aleti_INFO5731_Fall2024/blob/main/in_class_excercise_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In-Class Assignment — Data Preprocessing & Cleaning (Text)  
**Time:** 20 minutes  |  **Points:** 10  

## Instructions
- This is an individual in-class assignment.  
- Write your code **inside each answer cell**.  
- Print the required outputs.  
- Submit your GitHub/Colab link as instructed by the instructor.


You are given a small dataset of customer support messages as a **TAB-separated text file**:  
- `support_messages.txt`

You will download this file from **Canvas** and upload it to your **Google Colab** notebook.

**How to upload it to your Google Colab notebook?**

1. Download `support_messages.txt` from Canvas.
3. In **the left sidebar**, click the **Files** icon (folder).  
4. Click **Upload** and select `support_messages.txt`.

6. RightAfter uploading, the file will appear in the Colab file list on the left.

6. Right-click the file, copy its path, and paste it into the FILE_PATH variable in Q1.

7. Run Q1 to load the dataset.



> Important: Keep the file name exactly as `support_messages.txt`.


## Questions (Total = 10 points)

### Q1 (1 point) — Load the dataset
Load the TAB-separated file into a pandas DataFrame with columns: `id`, `message`.  
Print: **(a)** `df.shape`, **(b)** `df.head(3)`.

### Q2 (3 points) — Descriptive columns
Add these columns for each message and print the full DataFrame:
- `word_count`: number of words  
- `char_count`: number of characters  
- `num_count`: number of digits (0–9)  
- `upper_word_count`: number of ALL-CAPS words (e.g., `"WHY"`, `"DAMAGED"`)  

### Q3 (3 points) — Clean text
Build a `clean_text(text)` function and create a new column `clean` with these steps **in order**:
1) lowercase  
2) remove punctuation/symbols (keep letters/numbers/spaces)  
3) remove English stopwords (use **nltk** or **sklearn** list)  
4) remove extra spaces  

Print the **original** message and **clean** version for rows `id=1` and `id=4`.

### Q4 (2 points) — Regex extraction
Using RegEx, extract and create two new columns:
- `order_id`: first occurrence of pattern `ORD-####` (case-insensitive; `ord-1060` is valid)  
- `email`: first email address if present (otherwise `None`/`NaN`)  

Print: `id`, `order_id`, `email` for all rows.

### Q5 (1 point) — TF-IDF keywords
Using the `clean` column, compute **TF-IDF** for the messages and print the **top 5 keywords** with the highest **average TF-IDF** across documents.


In [ ]:
# Setup (run this cell first)
import re
import pandas as pd


## Q1 (1 point) — Answer below

In [9]:
# Q1 — ANSWER CELL

FILE_PATH = "/content/support_messages.txt"

import pandas as pd

# Load TAB-separated file using names (this includes header row as data → shape (9,2))
df = pd.read_csv(FILE_PATH, sep="\t", names=["id", "message"])

# Print shape and first 3 rows
print(df.shape)
print(df.head(3))

(9, 2)
   id                                            message
0  id                                            message
1   1  Hi!! My ORDER is late :(  Order# ORD-1042. Ema...
2   2  Refund please!!! I was charged 2 times... invo...


## Q2 (3 points) — Answer below

In [10]:
# Q2 — ANSWER CELL

# word_count
df["word_count"] = df["message"].apply(lambda x: len(str(x).split()))

# char_count
df["char_count"] = df["message"].apply(lambda x: len(str(x)))

# num_count (digits)
df["num_count"] = df["message"].str.count(r"\d")

# upper_word_count (ALL CAPS words)
df["upper_word_count"] = df["message"].apply(
    lambda x: sum(1 for token in str(x).split() if token.isupper())
)

# Print full DataFrame
print(df)

   id                                            message  word_count  \
0  id                                            message           1   
1   1  Hi!! My ORDER is late :(  Order# ORD-1042. Ema...          12   
2   2  Refund please!!! I was charged 2 times... invo...          11   
3   3        Great service, thanks! arrived in 2 days :)           8   
4   4  WHY is my package DAMAGED??? tracking says del...           8   
5   5  Need to change address: 7421 Frankford Rd Apt ...          12   
6   6  Support ticket: ORD-1050. Call me at (469) 555...           9   
7   7  I can’t login— password reset link not working...          10   
8   8  Item missing from box. pls send replacement!! ...          10   

   char_count  num_count  upper_word_count  
0           7          0                 0  
1          73          4                 2  
2          71          9                 3  
3          43          1                 0  
4          55          0                 2  
5        

## Q3 (3 points) — Answer below

In [11]:
# Q3 — ANSWER CELL

import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

STOPWORDS = set(ENGLISH_STOP_WORDS)

# Remove header row accidentally read as data
df = df[df["id"] != "id"].copy()

# Convert id column to integer
df["id"] = pd.to_numeric(df["id"], errors="coerce")

def clean_text(text: str) -> str:

    # Step 1: lowercase
    text = str(text).lower()

    # Step 2: remove punctuation/symbols
    text = re.sub(r"[^a-z0-9\s]", " ", text)

    # Step 3: remove stopwords
    tokens = text.split()
    tokens = [t for t in tokens if t not in STOPWORDS]

    # Step 4: remove extra spaces
    return " ".join(tokens)

# Create clean column
df["clean"] = df["message"].apply(clean_text)

# Print original and clean for id=1 and id=4
print(df[df["id"].isin([1, 4])][["id", "message", "clean"]])

   id                                            message  \
1   1  Hi!! My ORDER is late :(  Order# ORD-1042. Ema...   
4   4  WHY is my package DAMAGED??? tracking says del...   

                                               clean  
1  hi order late order ord 1042 email sara ali gm...  
4            package damaged tracking says delivered  


## Q4 (2 points) — Answer below

In [12]:
# Q4 — ANSWER CELL

# Extract order_id (ORD-####, case-insensitive)
df["order_id"] = df["message"].str.extract(r"(?i)\b(ORD-\d{4})\b")

# Extract email
df["email"] = df["message"].str.extract(
    r"([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})"
)

# Print required columns
print(df[["id", "order_id", "email"]])

   id  order_id                  email
1   1  ORD-1042     sara.Ali@gmail.com
2   2  ORD-1042                    NaN
3   3       NaN                    NaN
4   4       NaN                    NaN
5   5       NaN                    NaN
6   6  ORD-1050                    NaN
7   7       NaN  mehri.sattari@unt.edu
8   8  ord-1060                    NaN


## Q5 (1 point) — Answer below

In [8]:
# Q5 — ANSWER CELL

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(df["clean"])

terms = vectorizer.get_feature_names_out()

# Calculate average TF-IDF score for each term
avg_scores = np.asarray(X.mean(axis=0)).ravel()

# Get top 5 indices
top_idx = avg_scores.argsort()[::-1][:5]

# Print keywords AND scores
print("Top 5 keywords with average TF-IDF scores:")
for i in top_idx:
    print(f"{terms[i]} : {avg_scores[i]:.4f}")

Top 5 keywords with average TF-IDF scores:
ord : 0.1384
order : 0.0990
1042 : 0.0709
email : 0.0658
service : 0.0559


## Grading Checklist
- Q1: correct load + prints  
- Q2: correct counts  
- Q3: cleaning follows the required order + prints for id=1 and id=4  
- Q4: regex extraction works (case-insensitive `ORD-####` and emails)  
- Q5: prints 5 keywords + their scores (rounding is fine)
